In [2]:
import nltk

In [3]:
import keras

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [16]:
import pickle


In [4]:
import numpy as np
import random
import string

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bhusal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bhusal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bhusal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
with open('chatbot_dataset.txt', 'r', encoding='iso-8859-1') as file:
    data = file.read()
# Tokenize the data
tokens = nltk.word_tokenize(data.lower())


In [9]:
import json
intents = json.loads(open("intents.json").read()) 

In [14]:

words = [] 
classes = [] 
documents = [] 
ignore_letters = ["?", "!", ".", ","] 
for intent in intents['intents']: 
    for pattern in intent['patterns']: 
        # separating words from patterns 
        word_list = nltk.word_tokenize(pattern) 
        words.extend(word_list)  # and adding them to words list 
          
        # associating patterns with respective tags 
        documents.append(((word_list), intent['tag'])) 
  
        # appending the tags to the class list 
        if intent['tag'] not in classes: 
            classes.append(intent['tag']) 

In [15]:
word_list

['who', 'is', 'the', 'hod']

In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()
words=[lemmatizer.lemmatize(word) 
       for word in words if word not in ignore_letters]
words=sorted(set(words))

In [21]:
pickle.dump(words, open('words.pkl', 'wb')) 
pickle.dump(classes, open('classes.pkl', 'wb')) 

In [25]:
training = [] 
output_empty = [0]*len(classes) 
for document in documents: 
    bag = [] 
    word_patterns = document[0] 
    word_patterns = [lemmatizer.lemmatize( 
        word.lower()) for word in word_patterns] 
    for word in words: 
        bag.append(1) if word in word_patterns else bag.append(0) 
          
    # making a copy of the output_empty 
    output_row = list(output_empty) 
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row]) 
random.shuffle(training) 


# Ensuring the shape is uniform
train_x = np.array([entry[0] for entry in training])
train_y = np.array([entry[1] for entry in training])


In [26]:

train_x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [27]:
train_y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
model = Sequential() 
model.add(Dense(128, input_shape=(len(train_x[0]), ), 
                activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(len(train_y[0]),  
                activation='softmax')) 



e:\Code_Himalaya\chatbot\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [31]:
# compiling the model 
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, metrics=['accuracy']) 
hist = model.fit(np.array(train_x), np.array(train_y), 
                 epochs=200, batch_size=5, verbose=1) 
  

e:\Code_Himalaya\chatbot\venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0517 - loss: 3.6243 
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0905 - loss: 3.4729
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1558 - loss: 3.2526 
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2172 - loss: 3.0319
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3085 - loss: 2.6765
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3452 - loss: 2.4741
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4185 - loss: 2.2200
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4588 - loss: 2.0746
Epoch 9/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4862 - loss: 1.9301
Epoch 10/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6047 - loss: 1.6158
Epoch 11/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6095 - loss: 1.5032
Epoch 12/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accurac

In [32]:
model.save("chatbotmodel.h5", hist)